In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import json
import os

In [11]:
def find_contours(img):
    # Grayscale image 
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)

    # Find the contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Return an empty array if it did not find any contours
    if len(contours) == 0:
        return np.array([])
    
    # Create a usable list of points from the contours
    sub_contours = []
    shrinked_list = []

    for contour in contours:
        sub_contour_points = []
        for contour_points in contour:
            sub_contour_points.append(contour_points[0])
        sub_contours.append(sub_contour_points)

        # Shrink the list of contours 
        shrinked_sub_list = []

        for i in range(0, len(sub_contour_points)):
            if i % 2 == 0: shrinked_sub_list.append(sub_contour_points[i])
        shrinked_list.append(shrinked_sub_list)

    return np.array(shrinked_list)

In [3]:
class Mask(object):
    def __init__(self, file):
        self.name = file[0].split(' ')[1].split('.')[0]
        self.classification = self.name.split('_')[0]
        self.contours = find_contours(cv2.imread(file[0]))
        

class Image(object):
    def __init__(self, file):
        self.image = cv2.imread(file)
        self.name = file.split('/')[2]
        self.mask = self.get_mask()
        
    def get_mask(self):
        path = "./test_mask/mask_Object " + self.name.split(' ')[1]
        return Mask(glob.glob(path))

In [12]:
# Save the points to a json file
images = []

for file in glob.glob("./test_image/*.png"):
    images.append(Image(file))
    
data = {}

for image in images:
    region = {}
    
    region['name'] = image.mask.name
    region['class'] = image.mask.classification
    x_points = []
    y_points = []
    for points in image.mask.contours:
        points = np.array(points)
        for point in points.transpose()[0].tolist():
            x_points.append(point)
            
        for point in points.transpose()[1].tolist():
            y_points.append(point)
    
    region['all_points_x'] = x_points
    region['all_points_y'] = y_points
    
    image_information = {}
    height, width, _ = image.image.shape
    image_information['filename'] = image.name
    image_information['width'] = width
    image_information['height'] = height
    image_information['region'] = region
    
    data['image_' + image.name] = image_information
    
with open('test_image/annotations.json', 'w') as json_file:
    json.dump(data, json_file)

In [8]:
#For testing
import random

annotations = json.load(open(os.path.join("test_image/annotations.json")))
annotations = list(annotations.values())

annotation = annotations[0]

fin_img = cv2.imread("./test_image/" + annotation['filename'])

x_points = annotation['region']['all_points_x']
y_points = annotation['region']['all_points_y']
# print(len(y_points) == len(x_points))

color = random.choice([[0,0,255], [255, 0, 0], [0, 255, 0], [255, 255, 0], [0, 255, 255], [255, 0, 255]])

points = np.array([x_points, y_points]).transpose()

cv2.polylines(fin_img, [points], True, color, 2)

# Show
cv2.imshow(annotation['filename'], fin_img)

cv2.waitKey(0)
cv2.destroyAllWindows()